In [15]:
from sagemaker import Session
import sagemaker
from sagemaker import get_execution_role
from sagemaker.workflow.parameters import ( 
 ParameterInteger, 
 ParameterString, 
 ParameterFloat) 

In [16]:
# Define IAM role
role = get_execution_role()
processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [17]:
bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'payment'
file_name = 'payment.csv'

train_path = f"s3://{bucket_name}/{prefix_name}/train"
validation_path = f"s3://{bucket_name}/{prefix_name}/validation"
test_path = f"s3://{bucket_name}/{prefix_name}/test"

In [18]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role

In [19]:
# sess = Session()
# input_source = sess.upload_data(file_name, bucket=bucket_name, key_prefix=prefix_name)
# input_source

input_source = 's3://{}/{}/{}'.format(bucket_name, prefix_name, file_name)
input_source

's3://eliezerraj-908671954593-dataset/payment/payment.csv'

In [20]:
data_location = 's3://{}/{}/{}'.format(bucket_name,
                                       prefix_name,
                                       'requirements.txt')
data_location

's3://eliezerraj-908671954593-dataset/payment/requirements.txt'

In [21]:
framework_version = "1.0-1"
sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    role=role,
    instance_type="ml.m5.large",
    instance_count=1,
    base_job_name='mlops-payment-feature-eng'
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [22]:
processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(source=input_source,
                        destination="/opt/ml/processing/input"),
        ProcessingInput(source=data_location,
                        destination="/opt/ml/processing/input/req/")
    ],
    outputs=[
        ProcessingOutput(output_name="train", 
                         source="/opt/ml/processing/train",
                         destination=f"s3://{bucket_name}/{prefix_name}/output/train"),
        ProcessingOutput(output_name="validation",
                         source="/opt/ml/processing/validation",
                         destination=f"s3://{bucket_name}/{prefix_name}/output/validation"),
        ProcessingOutput(output_name="test",
                         source="/opt/ml/processing/test",
                         destination=f"s3://{bucket_name}/{prefix_name}/output/test")],
    code=f"feat-v02.py",  # <- this gets uploaded as /opt/ml/processing/input/code
)

INFO:sagemaker:Creating processing-job with name mlops-payment-feature-eng-2024-04-02-20-06-58-682


............................Collecting imbalanced-learn
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 7.4 MB/s eta 0:00:00
/opt/ml/processing/input/code/feat-v02.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["distance"][ind] = distance
datetime64[ns, pytz.FixedOffset(-180)]
train_data:  (26989, 13)
validation_data:  (7712, 13)
test_data:  (3856, 13)
Fraudulent transactions are 8.30% of the train_data.
Fraudulent transactions are 7.65% of the test_data.
Fraudulent transactions are 3.83% of thevalidation_data.
Fraudulent transactions AFTER SMOTE are 50.00% of the train_data.
## Processing completed. Exiting.

